In [2]:
from parse_deck import parse_deck

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [7]:
# parse articles from blog
blog_url = "https://www.houhou-news.com/"
driver = webdriver.Chrome()
driver.get(blog_url)

articles = driver.find_elements(By.CLASS_NAME, "entry-card-wrap.a-wrap.border-element.cf")
driver.close()

print(articles)
print(len(articles))


[<selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="35a3588f-ac8b-49b0-8da2-64ac89a9dd0b")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="ac49c152-64c3-42af-9ef5-9f9a4c20b659")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="04468183-27ab-443f-af8b-5f03bdab90af")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="851a8ff3-82fd-4c93-8fb2-d43ec0f62f13")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="19c20e86-fbe1-43fd-a109-9545c8492390")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="7822266e-89a2-4932-8273-9a8e93260ba9")>, <selenium.webdriver.remote.webelement.WebElement (session="27b13ce30260944286af1561490867e6", element="c1e35e86-311a-4435-96f3-bc739417

In [ ]:
# update deck info from articles
# decks = [
#     deck category 1: [
#         {
#             deck_code: "",
#         }, ...
#     ],
#     deck category 2:...
# ]

In [ ]:
# update deck info by code
    
# decks = [
#     deck category 1: [
#         {
#             deck_code: "",
#             pekomons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             no_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# ]